<a href="https://colab.research.google.com/github/lisa-smallbird/stable-diffusion-colab-note-books/blob/main/DreamBooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# これは何？

DreamBoothを利用して学習データを作成するツールです。

SD1.4/WifuDiffusion/TrinartStableDiffusionを元に追加学習可能です


# 概要

npaka さんの記事を参考に作成しました

https://note.com/npaka/n/n3b96d0b5d9c2

このノートブックの使い方は下記のnote記事を確認下さい

https://note.com/lisa_s/n/n5fb5468e8998


# 本体

In [ ]:
#@title 1.GPUのリセマラ
#@markdown * 12GB以上のメモリが出るまでGPUガチャを引き直してください
#@markdown   * OK→12GB以上
#@markdown   * NG→12GB未満
#@markdown * NGだった場合は、メニューの「ランタイム」→「ランタイムを接続解除して削除」を実行してください
#@markdown * 最近の傾向として、NGは出ないと思いますが念のためです


!nvidia-smi | grep -oE '/ *[0-9]*MiB' | grep -o '[0-9]*' | awk '{print $1 "MiB";if ($1>12000){print "OK!"}else{print "NG!"}}'

In [ ]:
#@title 2.パッケージのインストールと再起動
#@markdown エラーが表示されますが、そのまま進んでOKです。
# PILの不具合問題対応
# (注意)以下を実行したら一度ランタイムを再起動すること
!sudo apt-get install tesseract-ocr
!pip install pytesseract==0.3.9 OmegaConf
import os
os.kill(os.getpid(), 9) # This will crash Colab (required, everything will still be intact so dont worry)

### 3.パッケージ等の準備２

GoogleDriveの利用許可を求められるので、適当に許可してください

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')
! mkdir -p /content/drive/MyDrive/diffusion/user-data/learning/
# パッケージのインストール
!pip install huggingface_hub

### 4.huggingfaceへログイン

In [ ]:
#@markdown * hugginfaceのログイントークンを利用し、ログインします
#@markdown   * 参考: [画像生成AI「Stable Diffusion」を低スペックPCでも無料かつ待ち時間なしで使う方法まとめ - Gigazine](https://gigazine.net/news/20220824-stable-diffusion-google-colaboratory/)


from huggingface_hub import notebook_login

notebook_login()

### 5.学習実行！

設定を記入し、実行してください

3000ステップで1時間～1.5時間程度かかります。

データは自動でGoogleDriveに保存されます


In [ ]:
#@title 
#残りのパッケージインストール
!git clone https://github.com/ShivamShrirao/diffusers.git
!git clone https://github.com/yuuk128/diffusion_sample.git
! cp -f diffusion_sample/train_dreambooth.py /content/diffusers/examples/dreambooth/train_dreambooth_lisa.py 
!pip install accelerate==0.12.0
!pip install diffusers
!pip install transformers
!pip install --upgrade Pillow==9.2.0
!pip install bitsandbytes

#
dream_booth_base_model = "Nilaier/Waifu-Diffusers" #@param ["hakurei/waifu-diffusion", "Nilaier/Waifu-Diffusers","CompVis/stable-diffusion-v1-4", "naclbit/trinart_stable_diffusion_v2,diffusers-115k", "naclbit/trinart_stable_diffusion_v2,diffusers-95k", "naclbit/trinart_stable_diffusion_v2,diffusers-60k"] {allow-input: true}
#@markdown * 学習元のモデルを入れてください
#@markdown   * hakurei/waifu-diffusion → Waifu-Diffusion 1.2
#@markdown   * Nilaier/Waifu-Diffusers → Waifu-Diffusion 1.3
#@markdown ----
#

dream_booth_input_dir = "/content/drive/MyDrive/diffusion/user-data/learning/dq10" #@param {type:"string"}
#@markdown * 学習したい画像の保存先を指定してください
#@markdown * GoogleDriveの diffusion/user-data/learning/dq10 に画像を保存した場合は、/content/drive/MyDrive/diffusion/user-data/learning/dq10 を指定してください
#@markdown ----

model_file_type_label = "diffusers\u5F62\u5F0F" #@param ["diffusers形式", "StableDiffusion形式（ckpt形式）", "両方"]
ckpt_prune="Yes" #@param ["Yes", "No"]
#@markdown * 保存形式を選択して下さい
#@markdown   * diffusers形式 → 私の公開している他のColabBookで使えます
#@markdown   * StableDiffusion形式（ckpt形式） → NMKD Stable Diffusion GUI や StableDiffusion(AUTOMATIC1111)で利用できます
#@markdown     * ckpt_prune → Yesにすると、ckpt形式で出力する際に多少質は下がりますが、容量は格段に削減されます
#@markdown ----
model_file_type_mapping = {
    "diffusers形式": "diffusers",
    "StableDiffusion形式（ckpt形式）": "ckpt",
    "両方": "both",
}
model_file_type = model_file_type_mapping[model_file_type_label]

dream_booth_model_name = "girl" #@param {type:"string"}
#@markdown * 学習したい対象の単語を入れてください
#@markdown * 学習したいのが女の子のキャラなら、girl、自宅の猫ならcatみたいな感じです。
#@markdown ----
dream_booth_steps = 3000 #@param {type:"integer"}
#@markdown * 学習にかけるステップ数です
#@markdown * 学習画像が多い場合はステップ数も多いほうが良さそうです
#@markdown * 学習素材にもよりますが、WaifuDiffusionでキャラ学習なら、画像枚数×150～200程度が良さそうです
#@markdown   * 例: 画像枚数が20枚なら3000で試してみて、再現度が微妙だったら4000で試す
#@markdown   * TriartSDv2を使う場合や、学習対象がキャラでない場合は、最適なステップ数は違いそうです
#@markdown ----

dream_booth_new_model = "dq10_3000" #@param {type:"string"}
#@markdown * 英数字で好きな名称をつけてください。保存先のファイル名でのみ使います
! ls  -l {dream_booth_input_dir}

import glob
imgs= glob.glob(f"{dream_booth_input_dir}/*")
if len(imgs) == 0:
  raise Exception("画像ファイルが見つかりません。dream_booth_input_dir があっているか確認してください")

strs = dream_booth_base_model.split(",")
dream_booth_base_model_name = strs[0]
ex_param = ""
if len(strs)==1:
  revision = ""
  script_name = "train_dreambooth.py"
else:
  revision = strs[1]
  script_name = "train_dreambooth_lisa.py"
  ex_param  = f" --revision '{revision}' "

diffusers_model_raw_path = f"/content/models/{dream_booth_new_model}"

!accelerate launch /content/diffusers/examples/dreambooth/{script_name} \
  --pretrained_model_name_or_path="{dream_booth_base_model_name}" {ex_param} \
  --instance_data_dir={dream_booth_input_dir} \
  --output_dir={diffusers_model_raw_path} \
  --instance_prompt="a photo of sks {dream_booth_model_name}" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps={dream_booth_steps} \
  --use_8bit_adam
! mkdir -p /content/drive/MyDrive/diffusion/user-data/DreamBooth/

print(model_file_type)
if model_file_type == "both" or model_file_type == "ckpt":
  !git clone https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05 jachiam
  ckpt_raw_path = f"/content/models/{dream_booth_new_model}.ckpt"
  !python jachiam/convert_diffusers_to_sd.py --model_path {diffusers_model_raw_path} --checkpoint_path {ckpt_raw_path}
  if ckpt_prune == "Yes":
    !git clone https://github.com/prettydeep/Dreambooth-SD-ckpt-pruning.git prune
    !python prune/prune-ckpt.py --ckpt /content/models/{dream_booth_new_model}.ckpt
    ckpt_use_path = f"/content/models/{dream_booth_new_model}-pruned.ckpt"
  else:
    ckpt_use_path = f"/content/models/{dream_booth_new_model}.ckpt"
  ! mv -v {ckpt_use_path} /content/drive/MyDrive/diffusion/user-data/DreamBooth/{dream_booth_new_model}.ckpt

if model_file_type == "both" or model_file_type == "diffusers":
  ! mv -v {diffusers_model_raw_path} /content/drive/MyDrive/diffusion/user-data/DreamBooth/{dream_booth_new_model}


### 6.セッションの切断
セッションを繋いだままだとクレジットを消費するので切断

In [ ]:
from google.colab import runtime
runtime.unassign()

### 7.学習したデータを元にお絵かき

[こちらのノートブック](https://colab.research.google.com/drive/1YoRB2tB8TGGlt2ef4AX4z8KJ5Og6y-wg?usp=sharing)にて作成した学習データは利用できます。

他にも、Diffuserに対応したツールなら学習データは使えると思います。

## おまけ：既存のDiffusers形式モデルをckptに変換

1,2,3まで実行してから↓を実行して下さい

In [ ]:
#@title 
#残りのパッケージインストール
!git clone https://github.com/ShivamShrirao/diffusers.git
!git clone https://github.com/yuuk128/diffusion_sample.git
! cp -f diffusion_sample/train_dreambooth.py /content/diffusers/examples/dreambooth/train_dreambooth_lisa.py 
!pip install accelerate==0.12.0
!pip install diffusers
!pip install transformers
!pip install --upgrade Pillow==9.2.0
!pip install bitsandbytes

ckpt_prune="Yes" #@param ["Yes", "No"]
#@markdown     * ckpt_prune → Yesにすると、ckpt形式で出力する際に多少質は下がりますが、容量は格段に削減されます
#@markdown ----
diffusers_model_path = "/content/drive/MyDrive/diffusion/user-data/DreamBooth/dq10" #@param {type:"string"}
#@markdown * 元のdiffusersファイルを指定して下さい
#@markdown ----
dream_booth_new_model = "dq10" #@param {type:"string"}
#@markdown * 英数字で好きな名称をつけてください。保存先のファイル名でのみ使います
! ls  -l {dream_booth_input_dir}

!git clone https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05 jachiam
ckpt_raw_path = f"/content/models/{dream_booth_new_model}.ckpt"
!python jachiam/convert_diffusers_to_sd.py --model_path {diffusers_model_path} --checkpoint_path {ckpt_raw_path}
if ckpt_prune == "Yes":
  !git clone https://github.com/prettydeep/Dreambooth-SD-ckpt-pruning.git prune
  !python prune/prune-ckpt.py --ckpt /content/models/{dream_booth_new_model}.ckpt
  ckpt_use_path = f"/content/models/{dream_booth_new_model}-pruned.ckpt"
else:
  ckpt_use_path = f"/content/models/{dream_booth_new_model}.ckpt"
! mv -v {ckpt_use_path} /content/drive/MyDrive/diffusion/user-data/DreamBooth/{dream_booth_new_model}.ckpt



---

## このノートブックの取り扱い

* 複製・改変・拡散・動画やブログでの紹介・その他何でもOKです
* むしろ、誰かwikiとか作って、真面目に管理してくれると助かります
* その他問い合わせは [Twitter@LisaDQX](https://twitter.com/LisaDQX)へご連絡ください
